In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

Generate data from stable AR(1) process
---

In [ ]:
# Parameters

mu = 0.
beta_true = 0.9

noise_obs = 0.1
T = 128

In [ ]:
# Generate synthetic data
x_true = np.random.randn(T)*noise_obs
for t in range(1, T):
    x_true[t] += beta_true*x_true[t-1]

In [ ]:
plt.plot(x_true);

Edward model
---

In [ ]:
from edward.models import Normal, InverseGamma, PointMass

mu = Normal(mu=0., sigma=10.0)
beta = Normal(mu=0., sigma=2.0)

noise_proc = tf.constant(0.1) #InverseGamma(alpha=1.0, beta=1.0)
noise_obs = tf.constant(0.1) #InverseGamma(alpha=1.0, beta=1.0)

x = [0] * T
x[0] = Normal(mu=mu, sigma=10.0)  # fat prior on x
for n in range(1, T):
    x[n] = Normal(mu=mu + beta * x[n-1], sigma=noise_proc)

# use variance as zero to convert the list of x's to something Inference
# will understand
#y = Normal(mu=x, sigma=tf.constant(0.))


### MAP Inference

In [ ]:
print("setting up distributions")
qmu = PointMass(params=tf.Variable(0.))
qbeta = PointMass(params=tf.Variable(0.))
print("constructing inference object")
inference = ed.MAP({beta: qbeta, mu: qmu}, data={xt: xt_true for xt, xt_true in zip(x, x_true)})
print("running inference")
inference.run()

print("parameter estimates:")
print("beta: ", qbeta.value().eval())
print("mu: ", qmu.value().eval())

### Variational inference

In [ ]:
print("setting up variational distributions")
qmu = Normal(mu=tf.Variable(tf.random_normal([])), sigma=tf.nn.softplus(tf.Variable(tf.random_normal([]))))
qbeta = Normal(mu=tf.Variable(tf.random_normal([])), sigma=tf.nn.softplus(tf.Variable(tf.random_normal([]))))
#qmu = Normal(mu=tf.Variable(0.), sigma=tf.nn.softplus(tf.Variable(0.)))
#qbeta = Normal(mu=tf.Variable(0.), sigma=tf.nn.softplus(tf.Variable(0.)))
print("constructing inference object")
inference_vb = ed.KLqp({beta: qbeta, mu:qmu}, data={xt: xt_true for xt, xt_true in zip(x, x_true)})
print("running inference")
inference_vb.run()

print("parameter estimates:")
print("beta: ", qbeta.mean().eval(), qbeta.std().eval())
print("mu: ", qbeta.variance().eval(), qmu.std().eval())

### Compare with estimate from statsmodels

In [ ]:
from statsmodels.tsa import ar_model

ar1_sm = ar_model.AR(x_true)
res = ar1_sm.fit(maxlag=1, ic=None, trend='c')

In [ ]:
res.params